# QC

Conceptos previos, WIP

## Funciones lógicas

Implementación de funciones lógicas como transformaciones matriciales sobre el espacio completo de posibilidades.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def showmat(m):
    plt.imshow(m); plt.axis('off');

In [ ]:
bit = [0,1]

In [ ]:
def decode(bits):
    r = np.zeros(2**len(bits),int)
    r[int(''.join(reversed([str(b) for b in bits ])),2)] = 1
    return r

def encode(oneshot):
    x = np.argmax(oneshot)
    l = np.round(np.log2(len(oneshot))).astype(int)
    fmt = f"{{x:0{l}b}}"
    return list(reversed( [int(d) for d in fmt.format(x=x) ] ))

In [ ]:
decode([0,1,0,1])

In [ ]:
encode([[0,0,1,0]])

Esta es la matriz identidad de configuraciones de 4 bits. Hay $2^4=16$ configuraciones, y por tanto tenemos un matriz $16\times 16$. Pero debemos recorrer las variables en orden contrario para que la correspondencia cuadre con encode y decode. (Bit menos significativo primero).

In [ ]:
# Así nos sale la identidad permutada
ident = np.array([decode((x0,y0,x1,y1)) for x0 in bit for y0 in bit for x1 in bit for y1 in bit]).T
showmat(ident)     

In [ ]:
# Así queda bien
ident = np.array([decode((x0,y0,x1,y1)) for y1 in bit for x1 in bit for y0 in bit for x0 in bit]).T
showmat(ident)     

La composición de circuitos corresponde al producto de estas matrices. Tenemos que tener cuidado en el orden de las entradas y salidas. La regla es orden contrario en los for.

Vamos a usar unas utilidades para generar cómodamente las combinaciones:

In [ ]:
import itertools
def bits(n):
    return itertools.product(*[bit for _ in range(n)])

def Oper(l):
    return np.array([decode(x) for x in l]).T

Vamos a hacer un sumador de 2 bits con sumadores de un bit, donde el carry de uno es entrada del otro.

En esta primera versión generamos dos circuitos diferentes, y comprobamos que encajan bien.

In [ ]:
# Calculamos el primer bit y el carry, y pasamos sin modificar el segundo bit
# x0 x1 y0 y1 --> s0 c1 x1 y1
stage1 = Oper([( (x0+y0+0)%2, (x0+y0+0)//2, x1, y1) for y1,y0,x1,x0 in bits(4) ])
print(stage1.shape)
showmat(stage1)

In [ ]:
encode( stage1 @ decode([0,0,1,1]) )

In [ ]:
# Calculamos el segundo bit y el carry con el carry anterior y pasamos el s0 sin modificar
# s0 c1 x1 y1 --> s0 s1 c2
stage2 = Oper([ [s0,(x1+y1+c1)%2,(x1+y1+c1)//2] for y1,x1,c1,s0 in bits(4)])
print(stage2.shape)
showmat(stage2)

In [ ]:
encode( stage2 @ decode([0,1,1,1]) )

Combinamos ambos circuitos:

In [ ]:
adder2 = stage2 @ stage1
print(adder2.shape)
showmat(adder2)

Comprobamos que es igual que el circuito completo construido directamente:

In [ ]:
joint = Oper( [ [(x0+y0)%2,(x1+y1+c1)%2,(x1+y1+c1)//2] for y1,y0,x1,x0 in bits(4) for c1 in [(x0+y0)//2] ])
print(joint.shape)
showmat(joint)

Y lo probamos con algunas entradas:

In [ ]:
# 1 + 0 = 1
encode(joint @ decode([1,0,0,0]))

In [ ]:
# 3 + 3 = 6
encode(joint @ decode([1,1,1,1]))

In [ ]:
# 2 + 1 = 3
encode(joint @ decode([0,1,1,0]))

In [ ]:
# 2 + 2 = 4
encode(joint @ decode([0,1,0,1]))

La gracia está en combinar circuitos fijos, expandiendo entradas adecuadamente con tensor products.

In [ ]:
def tp(A,B):
    return np.vstack([np.hstack(x) for x in np.tensordot(A,B,axes=0)])

In [ ]:
def addbit(M,n,left=True):
    if n == 0: return M
    if left:
        r = tp([[1,1]], M)
    else:
        r = tp(M, [[1,1]])
    return addbit(r,n-1,left)

In [ ]:
adder1 =Oper( [[(a+b+c)%2,(a+b+c)//2] for c,b,a in bits(3)])
print(adder1.shape)
showmat(adder1)

Anque todavía no lo usaremos, podemos cambiar el orden de las variables, sale una matriz de permutación:

In [ ]:
#switch = np.array([decode([b,c,a]) for c in bit for b in bit for a in bit]).T
switch = Oper([(b,c,a) for c,b,a in bits(3)])
showmat(switch)

Aplicándola 3 veces tenemos la identidad. Da lugar a un subgrupo cíclico, creo.

In [ ]:
showmat(switch@switch@switch)

Pero por el momento vamos a hacerlo "a huevo" y luego ya haremos los cables que atraviesan y los cambios de orden con tensor products.

Aunque lo hagamos así, lo bonito es reutilizar el mismo circuito adder1 en los dos trozos, para que sea realmente composicional.

In [ ]:
stage1b = Oper( list(encode(adder1 @ decode((x0,y0,0))))+[x1,y1] for y1,y0,x1,x0 in bits(4) )
showmat(stage1b)

In [ ]:
stage2b = Oper([s0] + list(encode(adder1 @ decode((x1,y1,c1)))) for y1,x1,c1,s0 in bits(4) )
showmat(stage2b)

Se ve que son las mismas de antes.

## Incertidumbre

Por supuesto, la gracia de todo esto es manejar automáticamente la incertidumbre.

Tenemos que generar el vector one-shot, pero ahora repartiendo la probabilidad entre todos los estados, dependiendo de lo que se conozca sobre cada variable.

Lo hacemos con el mismo generador de posibilidades, multiplicando las probabilidades de los factores que existan en el problema, si todas las variables son independientes queda completamente factorizada, pero no hay ningún problema con dependencias. De hecho el resultado no será separable.

In [ ]:
def bernoulli(p):
    return {0:round(1-p,5), 1:p}

def p(a,b,c):
    return bernoulli(0.8)[a] * (1 if b==0 else 0) * 1/2

state = [ p(*bs) for bs in bits(3)]
state

Supongamos que quiero sumar dos números y un bit es "dudoso".

In [ ]:
state = [ (x0==0) * bernoulli(0.5)[x1] * (y0==1) * bernoulli(0.8)[y1] for y1,y0,x1,x0 in bits(4)  ]
state

In [ ]:
joint @ state

In [ ]:
for k,v in enumerate(adder2 @ state):
    if v >0:
        print(k,v)

Traducido, significa: si $x$ es 0 ó 2, con igual probabilidad, (x1 es completamente ambiguo), e $y$ es casi seguro 3 (con probabilidad 0.8), o si no 1 (el bit y1 tiene una pequeña probabilidad de no ser correcto, lo más probable es que su suma sea 3, aunque tampoco podemos descartar 5, y más raramente 1. Pero 0,2,4,6 son imposibles.

Podríamos muestrear esta distribución.

Por cierto, podemos ver el estado de la computación después de la primera etapa:

In [ ]:
# el format da en orden contrario a lo de arriba
print('y1 x2 c1 s0')
for k,v in enumerate(stage1 @ state):
    if v >0:
        print(f'{k:04b}',v)

Es decir, s0 es seguro 1 y el carry 0, y los bits más significativos que atravisan reparten su probabilidad de acuerdo con el estado inicial.

Estas matrices de transformación son [matrices estocásticas](https://en.wikipedia.org/wiki/Stochastic_matrix), transforman densidades de probabilidad en densidades de probabilidad. Son probabilidades condicionadas, Y por tanto, cada columna debe sumar 1.

Recordemos que producto matriz vector implementa la contracción P(y) = Sum P(y|x) P(x).

In [ ]:
showmat(stage1)

Los ejemplos anteriores son circuitos deterministas, por tanto las columnas no solo suman 1 sino que cada elemento de la base de estados de entrada produce sin ambiguedad una configuración de salida. Eso sí, es completamente normal que varios estados de entrada vayan al mismo de salida. Cada fila contiene las configuraciones que la activan.

Podemos analizar son eigensystem y svd estas matrices y se saca información interesante.

In [ ]:
np.linalg.eig(stage1)[0]

## Computación Reversible

Pero hay algo más interesante. Si la matriz tiene inversa significa que la computación se puede deshacer, del estado final se puede volver al de partida. La matriz de suma del ejemplo anterior claramente no es invertible, la operación de suma, como tal, no es invertible, a menos que nos las arreglemos para mantener las entradas, explícita o implícitameante en el resultado.

Afortunadamente existen juegos universales de puertas lógicas reversibles, lo cual implica que en principio se puede computar sin consumir energía. La que se haya consumido se recupera deshaciendo la operación.

## Computación Cuántica

El paso siguiente es implementar la operación del problema de Deutsch-Jozsa, y ver la diferencia entre probabilidades y amplitudes.

In [ ]:
# Walsh-Hadamard Gate

WH = np.array([[1, 1],
               [1,-1]])/np.sqrt(2)

# two WH gates in parallel for two bits
mix = tp(WH,WH)

def konst(x):
    return 1

def balanced(x):
    return 1 if x == 1 else 0

fun = balanced
#fun = konst

def xor(x,y):
    return 1 if x!=y else 0

# creates a reversible operation with an auxiliary input
rever = Oper([( x, xor(y, fun(x)) ) for y,x in bits(2) ])

# check the operation and the order of bits
for x,y in bits(2):
    xs, yf = encode(rever @ decode([x,y]))
    print (x,y, xs == x, yf == xor(y,fun(x)))

# with the identity in the auxiliary first bit remains uncertain
# the second bit is the solution: 0 = konst, 1 = balanced
# amps = tp(np.eye(2),WH) @ rever @ mix @ decode([0,1])

amps = mix @ rever @ mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2

print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
print(rever.shape)
showmat(rever)

In [ ]:
showmat(mix@rever@mix)

In [ ]:
amps = decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2
print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
amps = mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2
print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
probs = rever @ [0.25,0.25,0.25,0.25]
probs

## Esencia de la QFT en Shor

In [ ]:
x = np.zeros(256)
x[5::9] = 1
plt.plot(x);
plt.show()

f = np.fft.ifft(x)
#plt.plot(np.real(f))
#plt.plot(np.imag(f));
#plt.show()
plt.plot(abs(f));

No se puede medir en x porque cada observación vendría con uno distinto y no podríamos deducir el período. Tenerlos todos para la QFT sí considera la propiedad global.

Hay que ver lo de los convergentes.

Precioso.